In [10]:
%run __init__.py
%load_ext autoreload
%autoreload 2

In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/konstantin/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/konstantin/.local/lib/python3.6/site-packages/m

In [11]:
model_config_path = '../configs/model.yaml'
mode_config_path = '../configs/analyze.yaml'

model_name = 'NetVGG'
model_version = ''

mode = 'analyze'
dataset = 'megadepth'
gpu = '2'

experiment = create_experiment(model_config_path, mode_config_path, model_name, model_version,
                               mode, dataset, gpu, '../')

model
	         grid_size : 8
	   descriptor_size : 64
	        batch_norm : True
	soft_nms_kernel_size : 7
	   nms_kernel_size : 3
	             top_k : 256
	   checkpoint_name : 4_ems=0.2371161
criterion
	det
		   nms_kernel_size : 3
		             top_k : 256
		 gauss_kernel_size : 7
		       gauss_sigma : 0.5
		            lambda : 4.0
		      loss_version : 2
	det_conf
		            lambda : 1.0
	desc
		            margin : 1.0
		            lambda : 1.0
dataset
	analyze
		megadepth
			      dataset_root : /home/konstantin/datasets/MegaDepthDataset
			   scene_info_root : /home/konstantin/datasets/MegaDepthDataset/SceneInfo
			          csv_path : /home/konstantin/datasets/MegaDepthDataset/SceneInfo/analyze_megadepth.csv
			            height : 240
			             width : 320
			      to_grayscale : True
			           sources : True
loader
	analyze
		        batch_size : 1
		       num_samples : 1
		           shuffle : False
		       num_workers : 8
experiment
	        num_epochs

In [12]:
(batch, endpoint), metrics = experiment.run()

Model 4_ems=0.2371161 is loaded


/home/konstantin/.local/lib/python3.6/site-packages/torch/nn/functional.py:2479: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [15]:
image1 = batch.get(du.IMAGE1)
image2 = batch.get(du.IMAGE2)

score1 = endpoint[eu.SCORE1]
score2 = endpoint[eu.SCORE2]

kp1 = endpoint[eu.KP1]
kp2 = endpoint[eu.KP2]

w_kp1 = endpoint[eu.W_KP1]
w_kp2 = endpoint[eu.W_KP2]

w_vis_kp1_mask = endpoint[eu.W_VIS_KP1_MASK]
w_vis_kp2_mask = endpoint[eu.W_VIS_KP2_MASK]

desc1 = endpoint[eu.DESC1]
desc2 = endpoint[eu.DESC2]

kp1_desc = endpoint[eu.KP1_DESC]
kp2_desc = endpoint[eu.KP2_DESC]

shift_scale1 = batch.get(du.SHIFT_SCALE1)
shift_scale2 = batch.get(du.SHIFT_SCALE2)

intrinsics1 = batch.get(du.INTRINSICS1)
intrinsics2 = batch.get(du.INTRINSICS2)

extrinsics1 = batch.get(du.EXTRINSICS1)
extrinsics2 = batch.get(du.EXTRINSICS2)

In [20]:
from Net.source.nn.net.utils.criterion_utils import create_w_desc_grid, create_neigh_mask_ids
from Net.source.utils.eval_utils import draw_neigh_mask

desc_shape = desc2.shape

w_desc_grid1, w_vis_desc_grid_mask1, w_desc_grid2, w_vis_desc_grid_mask2 = create_w_desc_grid(image1, image2, batch, 8)

neigh_mask_ids = create_neigh_mask_ids(w_desc_grid1, desc_shape, 8)

cv_neight_mask = draw_neigh_mask(image2, w_desc_grid1[0], neigh_mask_ids[0], desc_shape, 8, 482)

plot_figures({'t': cv_neight_mask}, 1, 1, (18, 18))

IndexError: index 255 is out of bounds for dimension 0 with size 1

In [9]:
# s_image1 = batch[S_IMAGE1]
# s_image2 = batch[S_IMAGE2]

# kp1 = batch[KP1]
# kp2 = batch[KP2]

# w_kp1 = batch[W_KP1]
# w_kp2 = batch[W_KP2]

# score1 = batch[SCORE1]
# score2 = batch[SCORE2]

# wv_kp2_mask = batch[WV_KP2_MASK]
# wv_kp1_mask = batch[WV_KP1_MASK]

# desc1 = batch[DESC1]
# desc2 = batch[DESC2]

# w_vis_mask1 = batch[W_VIS_MASK1]
# w_vis_mask2 = batch[W_VIS_MASK2]

# debug1 = batch[DEBUG1]
# debug2 = batch[DEBUG2]

# homo12 = batch[HOMO12]
# homo21 = batch[HOMO21]

In [8]:
# # FOS 

# grid_size = 8
# margin = 1
# loss_lambda = 1

# b, c, hc, wc = desc1.size()
# flat = hc * wc

# coo_grid1 = create_desc_coordinates_grid(desc1, grid_size, False).to(desc1.device)
# w_coo_grid1 = warp_coordinates_grid(coo_grid1, homo12).view(b, -1, 2)
# w_coo_grid1 = w_coo_grid1[:, :, [1, 0]]

# w_desc1 = sample_descriptors(desc2, w_coo_grid1, grid_size)

# # Create neigh mask
# coo_grid2 = create_center_desc_coordinates_grid(desc2, grid_size).view(b, -1, 2).to(desc2.device)
# coo_dist = calculate_distance_matrix(w_coo_grid1, coo_grid2)
# _, ul = coo_dist.min(dim=-1)

# ul = ul.unsqueeze(-1)

# ur = ul + 1
# ur = torch.where(ur >= flat, ul, ur)

# ll = ul + wc
# ll = torch.where(ll >= flat, ul, ll)

# lr = ll + 1
# lr = torch.where(lr >= flat, ul, lr)

# mask_ids = torch.cat([ul, ur, ll, lr], dim=-1)

# neigh_mask = torch.zeros_like(coo_dist).to(coo_dist.device)
# neigh_mask = neigh_mask.scatter(dim=-1, index=mask_ids, value=1)

# # Prepare visibility mask
# w_vis_mask1 = space_to_depth(w_vis_mask1, grid_size)
# w_vis_mask1 = w_vis_mask1.prod(dim=1).unsqueeze(1).view(b, 1, -1)

# # Get positive and negative matches
# desc1 = desc1.view(b, c, -1).permute((0, 2, 1))
# desc2 = desc2.view(b, c, -1).permute((0, 2, 1))

# desc_sim = calculate_inv_similarity_matrix(desc1, desc2)
# desc_sim = desc_sim + neigh_mask * 5
# desc_sim = desc_sim + (1 - w_vis_mask1.float()) * 5

# wv_match_mask1 = get_visible_keypoints_mask(score2, w_coo_grid1).float()

# pos_sim = calculate_inv_similarity_vector(desc1, w_desc1)
# neg_sim = desc_sim.min(dim=-1)[0]

# loss = torch.clamp(pos_sim - neg_sim + margin, min=0) ** 2 * wv_match_mask1
# loss = loss.sum() / wv_match_mask1.sum() * loss_lambda

# print(loss)

In [7]:
# cv_s_image1 = torch2cv(s_image1.squeeze(0))
# cv_s_image2 = torch2cv(s_image2.squeeze(0))

# coo_grid1 = coo_grid1.view(b, -1, 2)[:, :, [1, 0]]

In [6]:
# cell_id = 2300

# image1_cell = coo_grid1[0, cell_id].unsqueeze(0)
# w_image1_cell = w_coo_grid1[0, cell_id].unsqueeze(0)

# w_image1_cells = flat2grid(neigh_mask[0, cell_id].nonzero(), wc).squeeze(1) * grid_size

In [5]:
# res_image1 = draw_cv_keypoints(cv_s_image1, image1_cell, (0, 255, 0))

# res_image2 = draw_cv_keypoints(cv_s_image2, w_image1_cell, (0, 255, 0))
# res_image2 = draw_cv_keypoints(res_image2, w_image1_cells, (0, 0, 255))

# plot_figures({'img1': res_image1,
#               'img2': res_image2}, 2, 1, (18, 18))

In [4]:
# # SOS
# sos_neg = 4

# cells1_mask = torch.eye(coo_grid1.size(1)).repeat(b, 1, 1)

# mask_ids = mask_ids.view(b, -1)

# w_cell1_dist = calculate_difference_matrix(mask_ids, mask_ids)
# w_cell1_mask = w_cell1_dist.eq(0).view(b, flat, 4, flat, 4).sum(-1).sum(-2).float()

# cells1_sim  = calculate_inv_similarity_matrix(desc1, desc1)
# cells1_sim = cells1_sim + cells1_mask * 5

# w_cells1_sim = calculate_inv_similarity_matrix(w_desc1, w_desc1)
# w_cells1_sim = w_cells_sim + w_cell1_mask * 5

# _, cells1_neg_ids = cells1_sim.topk(k=sos_neg, dim=-1, largest=False)
# _, w_cells1_neg_ids = w_cells1_sim.topk(k=sos_neg, dim=-1, largest=False)

# cells1_neg_ids = cells1_neg_ids.view(b, flat * sos_neg).unsqueeze(-1).repeat(1, 1, c)
# w_cells1_neg_ids = w_cells1_neg_ids.view(b, flat * sos_neg).unsqueeze(-1).repeat(1, 1, c)

# cells1_neg_desc = desc1.gather(dim=1, index=cells1_neg_ids)
# w_cells1_neg_desc = w_desc1.gather(dim=1, index=w_cells1_neg_ids)

# desc1 = desc1.unsqueeze(2).repeat(1, 1, sos_neg, 1).view(b, flat * sos_neg, c)
# w_desc1 = w_desc1.unsqueeze(2).repeat(1, 1, sos_neg, 1).view(b, flat * sos_neg, c)

# sos = calculate_inv_similarity_vector(desc1, cells1_neg_desc) - calculate_inv_similarity_vector(w_desc1, w_cells1_neg_desc)

# sos = (sos ** 2).view(b, flat, sos_neg).sum(-1).sqrt().mean()

# print(sos)

In [3]:
# w_image1_cell_mask = w_coo_grid1[0, w_cell1_mask[0, cell_id].nonzero()].squeeze(1)

In [2]:
# res_image1 = draw_cv_keypoints(cv_s_image1, image1_cell, (0, 0, 255))

# res_image2 = draw_cv_keypoints(cv_s_image2, w_image1_cell, (0, 0, 255))
# res_image2 = draw_cv_keypoints(res_image2, w_image1_cells, (0, 0, 255))
# res_image2 = draw_cv_keypoints(res_image2, w_image1_cell_mask, (255, 0, 255))


In [1]:
# plot_figures({'img1': res_image1,
#               'img2': res_image2}, 2, 1, (18, 18))